<h1>Pretrained MobileNetV2</h1>
<p>Additional documentation for pretrained model model can be found at <a href=https://github.com/rwightman/pytorch-image-models/blob/master/docs/models/mobilenet-v2.md>https://github.com/rwightman/pytorch-image-models/blob/master/docs/models/mobilenet-v2.md</a>.</p>

<h2>System Setup</h2>
<p>Run the follwing command in terminal to install the deep learning library:</p>

In [ ]:
pip install timm

<h2>Import Statements + Model Instantiation</h2>

In [ ]:
# Timing
import time

# Model
import timm
import torch
from matplotlib import pyplot as plt
from timm.data.transforms import RandomResizedCropAndInterpolation
from gc import enable

# Image Processing
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [ ]:
# Indicate device and model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
m = timm.create_model('mobilenetv2_050', pretrained=True)

# Set model into evaluation mode
m.eval()

<h2>Load + Preprocessing for a Single Image</h2>
<p>The prepocessing performs a random 224x224 crop because the test image should be unpredictable.</p>

In [ ]:
config = resolve_data_config({}, model=m)
transform = create_transform(**config)

# If fetching the test image from a url, use the format below:
# url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# urllib.request.urlretrieve(url, filename)

# Otherwise, mount Google drive to notebook and open image as a file:
img_path = '/content/drive/MyDrive/ENGINE: Wyze/Images/test/n02093428-American_Staffordshire_terrier/n02093428_0.jpg';
img = Image.open(img_path).convert('RGB')

# Randomly crop the image to 224x224
tfm = RandomResizedCropAndInterpolation(size=224)
random = tfm(img)
plt.imshow(random)

tensor = transform(img).unsqueeze(0) # transform and add batch dimension

<h2>Helper Function Definition</h2>
<ul><li>accuracy() : find the accuracy of the model for a single image. The function also incorporates timing evaluation calls.</li>

In [ ]:
def accuracy(img_path):
  img = Image.open(img_path).convert('RGB')

  # Randomly select a 512x512 frame
  tfm = RandomResizedCropAndInterpolation(size=512)
  random_crop = tfm(img)
 
  # Transform and add batch dimension
  tensor = transform(img).unsqueeze(0)

  # Start timer to calculate latency and make prediction w model
  latency_s = time.time()
  with torch.no_grad():
    out = m(tensor)
  latency = (time.time()-latency_s)*1000
 

  # Probabilities for each breed
  probabilities = torch.nn.functional.softmax(out[0], dim=0)

  # Grab breeds classes
  url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
  urllib.request.urlretrieve(url, filename)

  with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

  # Get top 1 for the image
  #   -top1_prob[0].item() is the probability
  #   -categories[top1_catid[i]] is the label
  top1_prob, top1_catid = torch.topk(probabilities, 1)

  confidence = top1_prob[0].item()
  label = categories[top1_catid[0]]
  return confidence, label, latency

<h2>Main() to Find Accuracy of Small Test Dataset</h2>
<p>For each image, the function will print out the predicted breed alongside the confidence. To find accuracy, manually confirm that the predicted breed matches the image class and evaluate (# of correct predictions/total # of images).</p>

In [ ]:
def main():
  # Path to folders containing dog images
  folders = ['/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02085620-Chihuahua',
  '/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02091032-Italian_greyhound',
  '/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02093428-American_Staffordshire_terrier',
  '/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02097047-miniature_schnauzer',
  '/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02099712-Labrador_retriever',
  '/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02106662-German_shepherd',
  '/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02107142-Doberman',
  '/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02110063-malamute',
  '/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02110185-Siberian_husky',
  '/content/drive/MyDrive/ENGINE: Wyze/Images/test_yt/n02113799-standard_poodle']

  # Size and label of each folder respectively
  size = [10, 7, 4, 10, 8, 11, 10, 8, 10, 10]
  labels = ['n02085620', 'n02091032', 'n02093428', 'n02097047', 'n02099712',
            'n02106662', 'n02107142', 'n02110063', 'n02110185', 'n02113799']

  # Ongoing sum of total images and total latency
  total_img = 0
  total_l = 0

  # Loop through directory
  for i in range(len(size)):
    curr_dir = folders[i]
    dir_size = size[i]
    print(curr_dir)

    # Loop through each file in a folder
    for j in range(dir_size):
      img_path = curr_dir + '/' + labels[i] + '_' + str(j) + '.jpg'
      print(img_path)
      confidence, label, latency = accuracy(img_path)
      total_l += latency
      print('    ' + label + '    ' + str(confidence))
      total_img += 1

  print(total_l/total_img)
  print()
  print(total_img)

if __name__ == '__main__':
  main()